# Financial model

### <u>Table of contents:</u>

**1. How to use this notebook**<br/>
**2. Background information**<br/>
**3. Building a financial model**<br/>
**4. Literature**<br/>


**1. How to use this notebook**

This is a tool for aspiring and current GaiaHub contributors who want to see an example of how the Digital Gaia engine can be used to model financial aspects.

In this notebook we will see how we used the engine to build a simple financial model of Roots & Culture's indoor hemp farm in Virginia. If you haven't seen the general model of the Roots and Culture Farm, you can take a look here: [Roots and Culture notebook](https://github.com/gaia-os/gaia-hub/blob/main/notebooks/LAB-roots-and-culture.ipynb).


**2. Backround information**

**Roots & Culture (RC)** is an indoor hemp farm in Virginia, USA where high CBD hemp flowers are grown, harvested, then processed into extract which is infused into skincare products. Since 2018, they have been growing indoors because it offers control over all aspects of the environment, ensuring plants experience only the most favourable conditions during development. The result is an extremely clean, high-value extract which is perfect for dermatological applications.

Again, if you haven't seen the general model of the Roots and Culture Farm, you can take a look here: [Roots and Culture notebook](https://github.com/gaia-os/gaia-hub/blob/main/notebooks/LAB-roots-and-culture.ipynb).

For years, **RC** have been collecting detailed environmental data (temperature, relative humidity, light and more) from [Pulse sensors](https://pulsegrow.com/products/pulse-one?variant=48139205901) in all their facilities. In addition to this data, they performed soil tests in all rooms before and after each grow, and have been collecting standard agronomic statistics (planting rate, harvest weights, etc.). 

The grow cycle we will use for model demos is a cycle that ran from June 24, 2019 to October 14, 2019 and is  preconfigured in the `roots-indoor1.json` configuration file, you can also check it out on the roots and culture notebook.


**3. Building a financial model**

In order to build a financial model, we worked with data of the farm as well as with literature. On the first place we worked with data provided by the roots and culture farm, and we build a simple model containg elements of labor and energy cost. In order to to this we added some classes on the ontology: CropLocation (indicates if the crop is located in the indoor or in the germination tent) and DayHours (indicates the number of hours of daylight that the crop receives).
In the roots_agent, we added parameters regarding energy and labor use and cost, as well as methods to calculate energy and labor costs. In addition, based on peer-reviewed literature, we add variables and methods intended to calculate the monetary cost of supplies as seeds and fertilizer. Finally, we use a variable to calculate the total monetary cost (seeds cost + fertilizer cost + energy cost + labor cost). This value will be modified as the financial model becomes more complex and includes new variables.

Now that we've made our changes to the agent file, we can predict financial aspects and relate them to other variables.
Here an example:

In [ ]:
# Import required packages
import os
import sys
import matplotlib.pyplot as plt
import jax.numpy as jnp
from numpyro.optim import optax_to_numpyro
from numpyro.infer import Trace_ELBO
import optax
from numpyro import set_platform

# Retrieve the fangorn directory (option 1)
module_path = os.path.abspath(os.path.join('../fangorn/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# # Retrieve the fangorn directory (option 2)
# import digital_gaia.fangorn as fangorn
# fangorn_dir = dirname(dirname(dirname(abspath(fangorn.__file__))))

# Tell numpyro whether to use cpu or gpu
use_gpu = False
if use_gpu is True:
    set_platform('gpu')
else:
    set_platform('cpu')
    
# Import required classes from the fangorn module
from digital_gaia.fangorn.agents.AgentFactory import AgentFactory
from digital_gaia.fangorn.assessment.DataLoader import DataLoader
from digital_gaia.fangorn.kernels.impl.MCMCKernel import MCMCKernel
from digital_gaia.fangorn.kernels.impl.SVIKernel import SVIKernel
from digital_gaia.fangorn.visualisation.distributions import draw_beliefs
from digital_gaia.fangorn.visualisation.distributions import compare_posteriors

**Load the configuration file and agent(s)**

In [ ]:
# Load the configuration file corresponding to the Roots & Culture model
data_loader = DataLoader('../fangorn/data/projects/Roots-and-Culture/roots-indoor1.json')

In [ ]:
# Load the agents agent(s) compatible with the loaded data
agents = AgentFactory.create(data_loader, verbose=True)

# Get ordinary differential equations (ODE) and stochastic differential equations (SDE) agents
ode_agent = next(filter(lambda a: a.name == "Roots-and-Culture.roots-indoor1.Deterministic", agents))

In [ ]:
# Predict the future using the ODE and SDE agents
ode_prediction_samples = ode_agent.predict(model=ode_agent.model, num_samples=1)

**Visualize the predictions**

In [ ]:
# Draw prior beliefs using the predictive samples
fig = draw_beliefs(
        ode_prediction_samples,
        var_1={
            "labor_monetary_cost": "Labor Monetary cost",
            "total_monetary_cost": "Total Monetary cost"
        },
        var_2={
            "energy_monetary_cost": "Energy Monetary cost",
            "seeds_monetary_cost": "Seeds Monetary cost"  
        },
        measured=[False, False],
        fig_size=(16, 8.5)
    )

The first graph shows the cumulative monetary cost of energy and labor. In both cases, a maximum value is reached when the harvest is reached and the cycle is completed. The cost of energy depends on the daylight hours and the location of the crop, while the cost of labor depends on the tasks performed each week. The second graph shows the total accumulated monetary cost, which in this case is simply the sum of the monetary cost of energy and labor. The cost of the fertilizer is not graphed because in this data set it was not fertilized at any time. The seed cost is constant (with a value of zero) because the seeds price data was not provided by the farm, so we left it fixed at zero, but if we had the value, an increased of seeds coat would be seen in the first week, which is the moment where the seeds were planted.

**4. Literature**

Finally here is some literature on financial models that was used to build the model and that may be helpful in expanding it:

Eaves, J., & Eaves, S. (2018). Comparing the Profitability of a Greenhouse to a Vertical Farm in Quebec. Canadian Journal of Agricultural Economics/Revue canadienne d'agroeconomie, 66(1), 43-54.

Maureira, F., Rajagopalan, K., & Stöckle, C. O. (2022). Evaluating tomato production in open-field and high-tech greenhouse systems. Journal of Cleaner Production, 337, 130459.